<h2>[Apple] Find Physical Devices</h2>

In [ ]:
from pathlib import Path
import json

PREFIX_PATH = Path("<USER_PREFERRED_DONWLOAD_PATH>/Apple/")
BASE_PATH = ""
REGISTERED_DEVICES_FILE_NAME = "Devices Registered with Apple Messaging.json"
ACCESSORY_DEVICE_INFO_FILE_NAME = "Other Data Part 1 of 2/Apple Features Using iCloud/BluetoothAccessory/AccessoryDeviceInfo.json"

trusted_device_file_path = PREFIX_PATH / BASE_PATH / REGISTERED_DEVICES_FILE_NAME
bluetooth_file_path = PREFIX_PATH / BASE_PATH / ACCESSORY_DEVICE_INFO_FILE_NAME

with open(trusted_device_file_path) as f:
    data = json.load(f)
print(data.get('devices')[0].get('os-version'))

#The next code lists all Bluetooth devices including their MAC addresses.
#CVE-2024-27867 enables an attacker to eavesdrop AirPods microphone using only a Mac address. Android devices using Apple products likely remain unpatched.
with open(bluetooth_file_path) as f:
    bt_data = json.load(f)

try:
    for accessory in bt_data.get('AccessoryDatabase')[0].get('devices'):
        name = accessory.get('Accessory Name')
        bt_mac_addr = accessory.get('Bluetooth Mac Address')
        print(name, bt_mac_addr)
except:
    print("No Bluetooth devices were found")

| **Output** |
| --- |
| iPhone OS,17.2.1,20C53 |
| AirPods Pro 4d:32:00:6a:42:d8 |

<h2>[Apple] Find ISP (Internet Service Provider) + Mobile Carrier Name</h2>

In [ ]:
from pathlib import Path
import pandas as pd
import re

PREFIX_PATH = Path("<USER_PREFERRED_DONWLOAD_PATH>/Apple/")
BASE_PATH = "Apple Media Services Information Part 1 of 2/Apple_Media_Services/Stores Activity/Other Activity/"
SUBSCRIPTION_CLICK_ACTIVITY_FILE_NAME = "Subscription Click Activity.csv"
ITUNES_PAYMENT_STACK_FILE_NAME = "iTunes Payment Stack - Activity.csv"

click_activity_file_path = PREFIX_PATH / BASE_PATH / SUBSCRIPTION_CLICK_ACTIVITY_FILE_NAME
payment_stack_file_path = PREFIX_PATH / BASE_PATH / ITUNES_PAYMENT_STACK_FILE_NAME

store_subscription_click_activity = pd.read_csv(click_activity_file_path)
itunes_payment_stack = pd.read_csv(payment_stack_file_path)

ip_company_column = itunes_payment_stack.get('IP Company','None')

def find_ip_company(ip_company_column):
    return next((value for value in ip_company_column if value != 'None'), None)

ip_company = find_ip_company(ip_company_column)
print(ip_company)

##### Mobile Carrier #####
def extract_carrier_value(entry):
    match = re.search(r'"carrier":"([^"]+)"', entry)
    print(match.group(1))
    return match.group(1) if match else None
    
for entry in store_subscription_click_activity['Page Details']:
    try:
        res = extract_carrier_value(entry)
        if(res != None):
            break
        print(res)
    except:
        pass


| **Output** |
| --- |
| Verizon |
| AT&T |

<h2>[Apple] Find All Developers Who Created the Apps on Your iOS Device (Top 10, Sorted by Date)</h2>

In [ ]:
from pathlib import Path
import pandas as pd
import re

PREFIX_PATH = Path("<USER_PREFERRED_DONWLOAD_PATH>/Apple/")
BASE_PATH = "Apple Media Services Information Part 1 of 2/Apple_Media_Services/Stores Activity/Account and Transaction History/"
STORE_TRANSACTION_FILE_NAME = "Store Transaction History.csv"
STORE_TRANSACTION_FREE_APPS_FILE_NAME = "Store Transaction History - Free Apps.csv"

store_transaction_file_path = PREFIX_PATH / BASE_PATH / STORE_TRANSACTION_FILE_NAME
store_transaction_free_file_path = PREFIX_PATH / BASE_PATH / STORE_TRANSACTION_FREE_APPS_FILE_NAME

paid_apps_df = pd.read_csv(store_transaction_file_path)
free_apps_df = pd.read_csv(store_transaction_free_file_path)

num_of_paid_app_providers = paid_apps_df['Seller'].drop_duplicates().count() or 0
paid_app_df_sorted_by_date = paid_apps_df.sort_values(by='Item Purchased Date', ascending=False)
top_10_paid_app_providers = paid_app_df_sorted_by_date['Seller'].drop_duplicates().head(10)

num_of_free_app_providers = free_apps_df['Seller'].drop_duplicates().count() or 0
free_app_df_sorted_by_date = free_apps_df.sort_values(by='Item Purchased Date', ascending=False)
top_10_free_app_providers = free_app_df_sorted_by_date['Seller'].drop_duplicates().head(10)


print(f"The user trusted {num_of_paid_app_providers} different paid app providers and {num_of_free_app_providers} free app providers.\n")

print(f"Here are the 10 latest paid providers:")
print(re.sub('(\n +|^ +)','\n',top_10_paid_app_providers.to_string(index=False)))

print(f"\nHere are the 10 latest free providers:")
print(re.sub('(\n +|^ +)','\n',top_10_free_app_providers.to_string(index=False)))

**Output**\
The user trusted 431 different paid app providers and 842 free app providers.

Here are the top 10 paid providers:

| **Seller** |
| --- |
| Audible, Inc. |
| JoyTunes |
| Apple Inc. |
| Sony Music |
| Microsoft Corporation |
| Netflix, Inc. |
| Spotify Ltd. |
| Adobe Inc. |
| Disney |
| Nintendo Co., Ltd. |

Here are the top 10 free providers:

| **Seller** |
| --- |
| Apple Inc. |
| Duolingo |
| Grammarly, Inc |
| Google LLC |
| OpenAI, L.L.C. |
| Meta Platforms, Inc. |
| Snap Inc. |
| Zoom Video Communications, Inc. |
| TikTok Inc. |
| Imangi Studios (known for ”Temple Run”) |

<h2>[Apple] Find the Top 3 Most Common Event Locations Based on Joe׳s Personal Calendar</h2>

In [ ]:
from pathlib import Path
import ics

PREFIX_PATH = Path("<USER_PREFERRED_DONWLOAD_PATH>/Apple/")
BASE_PATH = "iCloud Calendars and Reminders/Calendars/"

#Replace with the calendar file name
CALENDARS_FILE_NAME = "<CALENDAR_NAME>.ics"

calendar_file_path = PREFIX_PATH / BASE_PATH / CALENDARS_FILE_NAME

with open(calendar_file_path,'r') as f:
    icsFile = ics.Calendar(f.read())
    locations = [event.location for event in icsFile.events if event.location]
    location_df = pd.DataFrame(locations, columns=['Location'])
    print(location_df.value_counts().head(3))

| **Location** | **Count** |
| --- | --- |
| Shake Shack, 400 W 8th St, Los Angeles, CA 90014, United States | 9 |
| Corgi Cafe, C. de la Indústria, 78, Gràcia, 08025 Barcelona, Spain | 6 |
| South Jersey Sports Center, 100 Pike Rd Bldg C, Mt Laurel Township, NJ 08054, United States | 3 |

<h2>[Google] Find the Top 3 User Agents Used by Joe</h2>

In [ ]:
from pathlib import Path
import pandas as pd

PREFIX_PATH = Path("<USER_PREFERRED_DONWLOAD_PATH>/Google/")
BASE_PATH = "Takeout/Google Account/"
SUBSCRIBER_INFO_FILE_NAME = "<USERNAME>.SubscriberInfo.html"

path = PREFIX_PATH / BASE_PATH / SUBSCRIBER_INFO_FILE_NAME

user_agent_df = pd.read_html(path)
user_agent_table = user_agent_df[0]
top_3_user_agents_table = user_agent_table.groupby("Raw User Agents").count().sort_values("IP Address", ascending=False).head(3)
top_3_user_agents_table = top_3_user_agents_table.reset_index()[['Raw User Agents','Timestamp']]
top_3_user_agents_table.rename(columns={'Raw User Agents':'User Agent','Timestamp':'Count'},inplace=True)
print(top_3_user_agents_table)

**Output**

| User-Agents | Count |
| --- | --- |
| Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 | 8 |
| Mozilla/5.0 (iPhone; CPU iPhone OS 16_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Mobile/15E148 Safari/604.1 | 5 |
| Mozilla/5.0 (Linux; Android 13; SM-G991U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.136 Mobile Safari/537.36 | 3 |

<h2>[Google] Find Sensitive Data for Blackmail Purposes Using Past Searches </h2>

In [ ]:
from pathlib import Path
from bs4 import BeautifulSoup

PREFIX_PATH = Path("<USER_PREFERRED_DONWLOAD_PATH>/Google/")
BASE_PATH = "Takeout/My Activity/Search/"
MY_ACTIVITY_FILE_NAME = "My Activity.html"

my_activity_file_path = PREFIX_PATH / BASE_PATH / MY_ACTIVITY_FILE_NAME

sensitive_keyword_list = ["loan", "bankruptcy", "overdue", "debt", "credit score", "bailout", "budget deficit",
                          "arrears", "gambling", "betting", "Casino", "Poker", "Roulette"]

def read_large_html_file(file_path, chunk_size=1024):
    with open(file_path, 'r', encoding='utf-8') as file:
        while True:
            chunk = file.read(chunk_size)
            if not chunk:
                break
            
            soup = BeautifulSoup(chunk, 'html5lib')
            # Process the chunk
            for key in sensitive_keyword_list:
                if(key in soup.get_text()):
                    print(key , soup.get_text())

read_large_html_file(my_activity_file_path)

| **Output** |
| --- |
| ”loan”, You searched and visited “affordable loans for everyone!” |
| ”overdue” You searched and visited - “Your bill is overdue? Contact our lawyers now!” |

<h2> [Meta] Find user location</h2>

In [ ]:
from pathlib import Path
from bs4 import BeautifulSoup

#Replace the user׳s details
PREFIX_PATH = Path("<USER_PREFERRED_DONWLOAD_PATH>/Meta/")
BASE_PATH = "facebook-[firstnameLastname]-[exportDate]-XXXXXXXX/logged_information/location/"
PRIMARY_LOCATION_FILE_NAME = "primary_location.html"

primary_location_file_path = PREFIX_PATH / BASE_PATH / PRIMARY_LOCATION_FILE_NAME

#Depends if you enabled your gps setting on the device. If not, only IP and public info are considered.
with open(primary_location_file_path) as fp:
    soup = BeautifulSoup(fp, 'html5lib')

# Find the specific div that contains the location information
location_div = soup.find('div', class_='_2ph_ _a6-p')

# Extract the location text
location_text = location_div.get_text(separator=' ', strip=True).split()
for i, part in enumerate(location_text):
    part = part.strip("\u200f")
    if part.isdigit() and i > 0:
        user_location = ' '.join(location_text[i-1:i+2])
        break
if user_location:
    df = pd.DataFrame({'Location': [user_location]})
    print(df)
else:
    print("User location not found.")

**Output**

| Address | postalCode |
| --- | --- |
| **reduced address in US** | **reduced postal code** |

<h2>[Meta] Find the top 20 posts that drawn the most attention from the user </h2>

In [ ]:
from pathlib import Path
from bs4 import BeautifulSoup
import glob

PREFIX_PATH = Path("<USER_PREFERRED_DONWLOAD_PATH>/Meta/")
BASE_PATH = "facebook-[firstnameLastname]-[exportDate]-XXXXXXXX/download_data_logs/data_types/0_/"
DATA_LOGS_FILE_NAME = "page_*.html"

post_list = []
def get_all_files(pattern):
    return glob.glob(pattern)

def is_numeric(value):
    try:
        float(value)
        return True
    except (TypeError, ValueError):
        return False

file_pattern = PREFIX_PATH / BASE_PATH / DATA_LOGS_FILE_NAME
all_files = get_all_files(file_pattern)
for file in all_files:
    with open(file) as fp:
        soup = BeautifulSoup(fp, 'html5lib')
        table = soup.find('table')
        headers = [header.text.strip() for header in table.find_all('th')]
        rows = []
        for row in table.find_all('tr')[1:]:
            cells = row.find_all('td')
            row_data = {headers[i]: cells[i].text.strip() for i in range(len(cells))}
            if len(cells) > 2 and cells[2].a:
                content_url = cells[2].a['href'].split('/"><u>')[0]
                entry = {'Content': content_url, 'Time': cells[1].text.strip()}
                post_list.append(entry)

sorted_data = sorted(post_list, key=lambda x: float(x['Time']), reverse=True)
top_10_list = sorted_data[:10]

for entry in top_10_list:
    print(entry['Content'], entry['Time'])


**Output**

| **Content** | **secondsViewd** |
| --- | --- |
| https://www.facebook.com/groups/politicsForFun/permalink/1633432166653678/ | 3984.7 |
| https://www.facebook.com/loanMasterMoneyTalks/videos/332115653356/ | 3690.2 |
...